<center> <h1>Scor Hackathon - Team n°1</h1> </center>
<img src="img/both.png" width=600 /> 




<h1>UW Ratio preprocessing</h1> 

In this notebook, we will preprocess the ```UW_ratio``` dataset given by score. The UW Ratio, sometimes called Combined Ratio, is define as following:

$$ UW = \frac{\text{Incurred Losses+Expenses}}{\text{Earned Premiums}} $$


As it measures the incurred losses as well as expenses in relation to the total collected premiums for each particular contract, the premium price of new contracts should somehow related to the history of a particular company or to a particular contract insurance. In our case, we make the assumptions that missing values ```NA``` are simply null values in the original dataset.

We will preprocess the original dataset to obtain the following structure:
- For each contract number and year, we will give the UW ratio for every previous years (2006 - 2019).
- The resulting dataset is stored in the ```A - UW Ratio Preprocessing``` csv file.

In [2]:
!pip install tqdm

In [3]:
import pandas as pd
from tqdm import tqdm
import numpy as np

Please note that this notebook will be run by the Preporcessing notebook, where preprocess_train, df_t is defined

In [4]:
if preprocess_train:
    df = pd.read_excel('data/UW_Ratio.xlsx')
else:
    df = pd.read_excel('data/UW_Ratio_Test.xlsx')
contracts = df_t
contracts = contracts[['FACUL_NUM','UWYEAR']]

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21150 entries, 0 to 21149
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FACUL_NUM      21150 non-null  object 
 1   UWYEAR         21150 non-null  object 
 2   UW_RATIO_2014  10896 non-null  float64
 3   UW_RATIO_2015  12376 non-null  float64
 4   UW_RATIO_2016  14064 non-null  float64
 5   UW_RATIO_2017  16011 non-null  float64
 6   UW_RATIO_2018  17915 non-null  float64
 7   UW_RATIO_2019  19800 non-null  float64
 8   UW_RATIO_2020  21040 non-null  float64
dtypes: float64(7), object(2)
memory usage: 1.5+ MB


In [16]:
contracts.head(3)

,Unnamed: 0,FACUL_NUM,DIVISION_NUM,HAZARD,SEGMENT_LOB,UWYEAR,CT_PERIOD,MAINOCCUPANCY,SECTOR,BUSINESSUNIT,...,Both_price(M),PD_percent_loss (%),BI_percent_loss (%),Both_percent_loss (%),PD_percent_tiv (%),COVER_BI,GEO_MARKET_SEGMENT,UWYEAR_label,INCEPTION_month,WEIGHTED_HAZARD
0,0,02F062851,4,0.266864,Ppty Non Energy,2019,2.564949,Precious Metals Mines,BS Energy,BS Energy,...,0.0,0.0,0.0,0.0,0.0,True,Latin America,1,4,0.266864
1,1,11F008861,1,1.225395,Ppty Non Energy,2017,2.564949,Precious Metals Mines,BS Energy,BS Energy,...,0.0,0.0,0.0,0.0,0.0,True,Latin America,1,4,1.225395
2,2,02F062851,4,0.381841,Ppty Non Energy,2018,2.564949,Precious Metals Mines,BS Energy,BS Energy,...,0.0,0.0,0.0,0.0,0.0,True,Latin America,1,3,0.381841


In [17]:
def uw_dataset(df_uw, df_c, threshold=2006):
    
    #Initialize dataframe
    columns = ['FACUL_NUM', 'UWYEAR']
    for year in list(np.arange(2006,2020)):
                  columns.append(year)
    data = []
    data = pd.DataFrame(data, columns = columns) 
    
    for c in tqdm(df_c['FACUL_NUM'].unique()) :
        years = df_c[ df_c['FACUL_NUM'] == c ]['UWYEAR'].unique()
        
        for year in years:
            year_text = 'UW_RATIO_' + str(year-1)
            df_uw['UWYEAR'] = df_uw.UWYEAR.str.replace(' ', '')
            uw_ratios = df_uw[ df_uw['FACUL_NUM'] == c][['UWYEAR', year_text]]
            uw_ratios = pd.DataFrame(uw_ratios[uw_ratios['UWYEAR'].astype('int64') < year ])
            uw_ratios = pd.DataFrame(uw_ratios[ (uw_ratios['UWYEAR'].astype('int64') < year) & (uw_ratios['UWYEAR'].astype('int64') > threshold) ])
            uw_ratios['FACUL_NUM'] = c
            uw_ratios['UWYEAR'] = uw_ratios['UWYEAR'].astype('int64')
            result =  uw_ratios.pivot_table(year_text, ['FACUL_NUM'], 'UWYEAR', aggfunc='first').reset_index()
            result['UWYEAR'] = year
            result.columns.name=None
            
            if result.empty:
                result= pd.DataFrame()
                result['UWYEAR'] = [year-1]
                result['FACUL_NUM'] = [c]
                
            data = pd.concat([data, result], join = 'outer')

    return data
    

In [18]:
df.UWYEAR = df.UWYEAR.astype('str')   

In [19]:
data = uw_dataset(df, contracts)

100%|██████████| 3017/3017 [02:46<00:00, 18.13it/s]


In [20]:
data = data.fillna(0.)
data.columns = data.columns.astype(str)
data.head()

,FACUL_NUM,UWYEAR,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,02F062851,2019,0.0,0.0,0.092736,4.653078,0.0,1.491216,0.598253,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
0,02F062851,2018,0.0,0.0,0.092736,4.653078,0.0,1.491216,0.598253,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
0,02F062851,2017,0.0,0.0,0.092736,4.653078,0.0,1.491216,0.598253,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
0,02F062851,2020,0.0,0.0,0.092736,4.653078,0.0,1.491216,0.598253,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
0,11F008861,2017,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.005662,0.0,0.0,0.0,0.0,0.0
